In [1]:
import generator_object, write_run_plot, serial_sim_tools
import numpy as np

from importlib import reload
for obj in [generator_object, write_run_plot, serial_sim_tools]:
    reload(obj)

In [2]:
def remove(name):
    import os, shutil
    path_rem = f'/home/brewster/Desktop/cretin_package-master/Personal_experiments/{name}/images'

    if os.path.isdir(path_rem):
        print(f'\nremoving {path_rem}')
        shutil.rmtree(path_rem)

In [5]:
import search
reload(search)

search_for_word = 'spectrum'

df = search.Generators().word(input = search_for_word, word_slicing = (0,5), print_path = True)
print(df.to_string())


search_inside_chapter = 'Switches and Parameters'
#print(search.Generators().chapter(input = search_inside_chapter))

/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//spec1d/spec1d.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//spec1d/spec1d.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//spec1d/spec1d.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//spec1d/spec1d.gen
/home/brewster/Desktop/creti

In [4]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(n, l):

    gen = generator_object.User_input()
    
    rho, T_ev = 1, 0.5
    z = 12. #
    N0, N1 = 1, 500  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = 1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 1*1.06 micrometer

    gen.materials_atom(index = 1, element = "sn")
    gen.materials_atom_isorange('all', n_max = n, lmax = l)
    gen.materials_atom_modeltype(['dca'])
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen

import time
times = []
for n in range(5,6):
    for l in range(2, n):

        sim4 = lasfoam_tin(n, l)
        name = f'Howard_scott36_{n}_{l}'
        #write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
        #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
        before  = time.time()
        #write_run_plot.run(name = name, object = sim4, longprint=False)
        after = time.time()
        print(f'{n} {l} {(after - before)/3600} hours')
        times.append([n, l , after-before])
        #write_run_plot.plot(name = name, object = sim4, longprint=False)
        #write_run_plot.extra_plot(name = name, make_animation=True)

5 2 1.3245476616753472e-10 hours
5 3 1.3245476616753472e-10 hours
5 4 1.3245476616753472e-10 hours
